# Saarland

Every federal state is represented by its own input directory and is processed into a NUTS level 2 directory containing a sub-folder for each discharge location. These folder names are derived from NUTS and reflect the CAMELS id. The NUTS level 2 code for Saarland is `DEC`.

To pre-process the data, you need to write (at least) two functions. One should extract all metadata and condense it into a single `pandas.DataFrame`. This is used to build the folder structure and derive the ids.
The second function has to take an id, as provided by the state authorities, called `provider_id` and return a `pandas.DataFrame` with the transformed data. The dataframe needs the three columns `['date', 'q' | 'w', 'flag']`.

For easier and unified output handling, the `camelsp` package contains a context object called `Bundesland`. It takes a number of names and abbreviations to identify the correct federal state and returns an object that holds helper and save functions.

The context saves files as needed and can easily be changed to save files with different strategies, ie. fill missing data with NaN, merge data into a single file, create files for each variable or pack everything together into a netcdf.

In [1]:
import pandas as pd
import numpy as np
from pandas.errors import ParserError
import os
from pprint import pprint
from tqdm import tqdm
from typing import Union, Dict
from datetime import datetime as dt
from dateparser import parse
import warnings

from camelsp import Bundesland

The context can also be instantiated as any regular Python class, ie. to load only the default input data path, that we will user later.

In [2]:
# the context also makes the input path available, if camelsp was install locally
BASE = Bundesland('saarland').input_path
BASE

'/home/alexander/Github/camels/camelsp/input_data/SL_Saarland'

### Metadata reader

Define the function that extracts / reads and eventually merges all metadata for this federal state. You can develop the function here, without using the Bundesland context and then later use the context to pass extracted metadata. The Context has a function for saving *raw* metadata, that takes a `pandas.DataFrame` and needs you to identify the id column.
Here, *raw* refers to provider metadata, that has not yet been transformed into the CAMELS-de Metadata schema.

In [3]:
# define the function 
def read_meta(base_path) -> pd.DataFrame:
    path = os.path.join(base_path, 'Qry_Pegel-Stammdaten.xlsx')
    meta = pd.read_excel(path)
    return meta

# test it here
metadata = read_meta(BASE)

metadata.head()

,MSTNR,MSTBEM,Pegelname_,Gewässer,Betreiber,Stromgebiet,GebkNR,EZG_Gr,Flusskm,PNP,RW,HW,HWMH_1,HWMH_2
0,1271120,Schieferstollen,Schieferstollen,Wadrill,LUA Saarland,Rhein/Saar/Prims/Wadrill,2646471,"44,200km²","9,71 km","344,48 m über NN",2563271,5495766,NaN,NaN
1,1122120,Geislautern,Geislautern,Rossel,LUA Saarland,Rhein/Mosel/Saar/Rossel,2644791,203km²,"3,275 km","184,38 m über NN",2560492,5455327,210 cm,250 cm
2,1482120,Lisdorf,Lisdorf,Saar,WSA Saarbrücken,Rhein/Mosel/Saar,2645599,4671km²,"65,885","173,50 m ü.NN",2555659,5462689,NaN,NaN
3,1251120,Gonnesweiler,Gonnesweiler,Bos,LUA Saarland,Rhein/Nahe/Bos,25411169,"12,500km²","0,08 km","372,40 m über NN",2579042,5492542,NaN,NaN
4,1071120,Nonnweiler IV,Nonnweiler IV,Prims,LUA Saarland,Rhein/Saar/Prims,2646149,"48,40km²","50,050 km","378,37 m über NN",2570340,5496980,NaN,NaN


In [4]:
# the id column will be MSTNR
id_column = 'MSTNR'

## file extract and parse

In [5]:
# build the filename
variable = 'w'
ID = metadata.loc[55, id_column]

def extract_file(ID: Union[int, str], variable: str, base_path: str) -> pd.DataFrame:
    # use always str ids and variable is used uppercase here
    ID = str(ID)
    sym = variable.upper()

    # build the path
    path = os.path.join(base_path, f'TM{sym}', f"{ID}-{sym}.TM{sym}")

    # if the files does not exist, return empty dataframe
    if not os.path.exists(path):
        return pd.DataFrame(columns=['date', variable.lower(), 'flag'])

    # read id
    df = pd.read_csv(path, encoding='latin1', sep='\s+', skiprows=1, header=None, parse_dates=[0], dayfirst=True, decimal=',', na_values=[9999, -9999])
    df.columns = ['date', variable.lower()]
    df['flag'] = np.NaN

    return df

# test stuff
extract_file(ID, variable, BASE)

,date,w,flag
0,2011-09-01,NaN,NaN
1,2011-09-02,NaN,NaN
2,2011-09-03,NaN,NaN
3,2011-09-04,NaN,NaN
4,2011-09-05,NaN,NaN
...,...,...,...
3770,2021-12-27,33.0,NaN
3771,2021-12-28,41.0,NaN
3772,2021-12-29,40.0,NaN
3773,2021-12-30,41.0,NaN


### Finally run

Now, the Q and W data can be extracted. The cool thing is, that all the id creation, data creation, merging and the mapping from our ids to the original ids and files is done by the context. This is helpful, as we less likely screw something up.

In [6]:
with Bundesland('Saarland') as bl:
    # save the metadata
    bl.save_raw_metadata(metadata, id_column, overwrite=True)

    # for reference, call the nuts-mapping as table
    nuts_map = bl.nuts_table
    print(nuts_map.head())

    
    with warnings.catch_warnings(record=True) as warns:
        for provider_id in tqdm(metadata[id_column].values.astype(str)):
            
            # get q and w
            q_df = extract_file(provider_id, 'q', bl.input_path)
            w_df = extract_file(provider_id, 'w', bl.input_path)

            bl.save_timeseries(q_df, provider_id)
            bl.save_timeseries(w_df, provider_id)

        # check if there were warnings (there are warnings)
        if len(warns) > 0:
            log_path = bl.save_warnings(warns)
            print(f"There were warnings during the processing. The log can be found at: {log_path}")


    nuts_id provider_id                              path
0  DEC10000     1271120  ./DEC/DEC10000/DEC10000_data.csv
1  DEC10010     1122120  ./DEC/DEC10010/DEC10010_data.csv
2  DEC10020     1482120  ./DEC/DEC10020/DEC10020_data.csv
3  DEC10030     1251120  ./DEC/DEC10030/DEC10030_data.csv
4  DEC10040     1071120  ./DEC/DEC10040/DEC10040_data.csv


100%|██████████| 56/56 [00:04<00:00, 11.33it/s]
